This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# To allow pushing the code as is without including the API key, 
# I put the API key in a separate file

with open('../quandl_key.txt', 'r') as f:
    quandl_key = f.read()

In [2]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = quandl_key

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [3]:
# First, import the relevant modules

import requests

**NOTE: I have added a function "make_url" below to generate the URL for the API call.**

In [4]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

database = 'FSE'
dataset = 'AFX_X'
startdate = '2014-01-05'
enddate = '2014-01-06'

def make_url(startdate, enddate, database=database, dataset=dataset):
    url = 'https://www.quandl.com/api/v3/datasets/' + database + '/' + dataset
    url += '/data.json?' + 'start_date=' + startdate + '&end_date=' + enddate
    url += '&api_key=' + API_KEY
    return(url)

r = requests.get(make_url(startdate=startdate,enddate=enddate))

In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
data = r.json()
type(data)

dict

In [6]:
# Top level of JSON object
[(k, type(data[k])) for k in data.keys()]

[('dataset_data', dict)]

In [7]:
# Second level of JSON object
[(k, type(data['dataset_data'][k])) for k in data['dataset_data'].keys()]

[('limit', NoneType),
 ('transform', NoneType),
 ('column_index', NoneType),
 ('column_names', list),
 ('start_date', str),
 ('end_date', str),
 ('frequency', str),
 ('data', list),
 ('collapse', NoneType),
 ('order', NoneType)]

In [8]:
# Third level of JSON object
[(k, data['dataset_data'][k]) for k in data['dataset_data'].keys()]

[('limit', None),
 ('transform', None),
 ('column_index', None),
 ('column_names',
  ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover']),
 ('start_date', '2014-01-05'),
 ('end_date', '2014-01-06'),
 ('frequency', 'daily'),
 ('data',
  [['2014-01-06',
    24.5,
    24.68,
    24.23,
    24.3,
    None,
    20469.0,
    500796.0,
    None,
    None,
    None]]),
 ('collapse', None),
 ('order', None)]

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### Task 1

In [9]:
# RESULT 1 (Collect data)
r = requests.get(make_url(startdate='2017-01-01', enddate='2017-12-31'))

### Task 2

In [10]:
# RESULT 2 (Convert to Python dictionary)
data_dict = r.json()
type(data_dict)

dict

### Explore data

In [11]:
# Top level of JSON object
[(k, type(data_dict[k])) for k in data.keys()]

[('dataset_data', dict)]

In [12]:
# Second level of JSON object
d = data_dict['dataset_data']
[(k, type(d[k])) for k in d.keys()]

[('limit', NoneType),
 ('transform', NoneType),
 ('column_index', NoneType),
 ('column_names', list),
 ('start_date', str),
 ('end_date', str),
 ('frequency', str),
 ('data', list),
 ('collapse', NoneType),
 ('order', NoneType)]

In [13]:
# "column_names" element to be used to identify series in the function below
d['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

**NOTE: I use the function below in subsequent tasks**

In [14]:
# Function to extract a data series given the column name
def get_column(name):
    idx = d['column_names'].index(name)
    return([x[idx] for x in d['data']])

### Task 3

In [15]:
# Get the series of opening prices
opens = get_column('Open')
len(opens)

255

In [16]:
# Some opening prices are missing, so just use the non-missing ones
available_opens = [x for x in opens if x]
len(available_opens)

252

In [17]:
# RESULT 3 (Highest and lowest opening prices)
min(available_opens), max(available_opens)

(34.0, 53.11)

### Task 4

In [18]:
# Get the two series for daily high and daily low prices
highs = get_column('High')
lows = get_column('Low')
len(highs), len(lows)

(255, 255)

In [19]:
# Calculate daily ranges for days when there is data for both high and low
#   (which turns out to be always)
available_ranges = [(hi,lo) for (hi,lo) in zip(highs,lows) if hi and lo]
len(available_ranges)

255

In [20]:
# RESULT 4 (Largest change in any one day)
max([hi-lo for (hi,lo) in available_ranges])

2.8100000000000023

### Task 5

In [21]:
# Get series of closing prices
closes = get_column('Close')
len([c for c in closes if c])

255

In [22]:
# RESULT 5 (Largest change in closing price between any two days)
diffs = [cur-last for (cur,last) in zip(closes[1:],closes[:-1])]
max(diffs)

2.559999999999995

### Task 6

In [23]:
vols = get_column('Traded Volume')
len([v for v in vols if v])

255

In [24]:
# RESULT 6 (Average daily trading volume)
sum(vols)/float(len(vols))

89124.33725490196